In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../datasets/glass.data.txt")
df = df.drop(columns=['Id'])
df.describe()

,Refractive_index,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
count,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,1.518365,13.407850,2.684533,1.444907,72.650935,0.497056,8.956963,0.175047,0.057009,2.780374
std,0.003037,0.816604,1.442408,0.499270,0.774546,0.652192,1.423153,0.497219,0.097439,2.103739
min,1.511150,10.730000,0.000000,0.290000,69.810000,0.000000,5.430000,0.000000,0.000000,1.000000
25%,1.516523,12.907500,2.115000,1.190000,72.280000,0.122500,8.240000,0.000000,0.000000,1.000000
50%,1.517680,13.300000,3.480000,1.360000,72.790000,0.555000,8.600000,0.000000,0.000000,2.000000
75%,1.519157,13.825000,3.600000,1.630000,73.087500,0.610000,9.172500,0.000000,0.100000,3.000000
max,1.533930,17.380000,4.490000,3.500000,75.410000,6.210000,16.190000,3.150000,0.510000,7.000000


In [2]:
#Maps a list of continuous values to discrete values given an interval
def discretize_column(dataframe_column, intervals):
    c_cut = pd.cut(dataframe_column ,intervals)
    intervals_dict = {intervals: index for index, intervals in enumerate(c_cut.cat.categories)}
    return [intervals_dict[value] for value in c_cut]


In [3]:
def write_arff(df, filename, intervals):

    def write_relation(file, relation):
        file.write("@RELATION {}\n\n".format(relation.lower()))
    
    def write_attribute(file, attr, possible_values):
        file.write("@ATTRIBUTE {} {{{}}}\n".format(attr.lower(), ",".join(map(str,possible_values))))
        
    def write_data(file, df, intervals):
        file.write("\n@DATA\n")
        mapped_columns = []
        #For values that need to be mapped
        for column in df.drop(columns=["Type"]):
            mapped_columns.append(discretize_column(df[column],intervals))
        #Type column is already discrete
        mapped_columns.append(list(df["Type"]))
        file.write(pd.DataFrame(mapped_columns).T.to_csv(header=False,index=False))

        
    file = open(filename,"w")

    #@RELATION
    write_relation(file,"Type")
      
    #@ATTRIBUTE
    for column in df.drop(columns=["Type"]):
        write_attribute(file, column, range(intervals))        
    write_attribute(file, "Type", range(8)[1:])
             
    #@DATA 
    write_data(file, df, intervals)
    
    file.close()

In [4]:
write_arff(df,"../datasets/crystal.arff",20)